In [16]:
from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


from warnings import filterwarnings
filterwarnings('ignore')

In [17]:
import pandas as pd 
data = pd.read_csv("train1.tsv", sep = "\t")

In [18]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


sentiment kısmındaki 0 ve 1 negatif yorumlar,2 orta düzey,3 ve 4 te pozitif yorumlar

In [19]:
data["Sentiment"].replace(0, value = "negatif",inplace = True)
data["Sentiment"].replace(1, value = "negatif",inplace = True)

In [20]:
data["Sentiment"].replace(3, value = "pozitif",inplace = True)
data["Sentiment"].replace(4, value = "pozitif",inplace = True)

2 yani orta kısmı kaldırıp tamamen binary hale getirelim

In [21]:
data = data[(data.Sentiment == "negatif") | (data.Sentiment == "pozitif")]
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negatif
21,22,1,good for the goose,pozitif
22,23,1,good,pozitif
33,34,1,"the gander , some of which occasionally amuses...",negatif
46,47,1,amuses,pozitif


dağılıma bakalım

In [22]:
data.groupby("Sentiment").count()

,PhraseId,SentenceId,Phrase
Sentiment,,,
negatif,34345,34345,34345
pozitif,42133,42133,42133


daha uygun şekilde değiştirelim ve anlaşılır hale getirelim


In [23]:
df = pd.DataFrame()
df["text"] = data["Phrase"]
df["label"] = data["Sentiment"]

Metin önişleme işlemlerini gerçekleştirelim

In [24]:
#buyuk-kucuk donusumu
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#noktalama işaretleri
df['text'] = df['text'].str.replace('[^\w\s]','')
#sayılar
df['text'] = df['text'].str.replace('\d','')
#stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
#seyreklerin silinmesi
sil = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))
#lemmi
from textblob import Word
nltk.download('wordnet')
df['text'] = df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Özellik Çıkarımı**

* Count Vectors 
* TF-IDF Vectors(words, characters, n-grams)
* Word embedding

In [25]:
df.head()

,text,label
0,series demonstrating adage good goose also goo...,negatif
21,good goose,pozitif
22,good,pozitif
33,gander occasionally amuses none amount much story,negatif
46,amuses,pozitif


**test train olarak ayıralım**

In [26]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df["text"],
                                                                     df["label"])

labellarımızı numerik hale getirelim

In [27]:
encoder = preprocessing.LabelEncoder()

In [28]:
train_y[0:10]

48492     pozitif
95922     pozitif
54821     negatif
24263     pozitif
91861     negatif
106265    pozitif
29327     negatif
134244    pozitif
86758     pozitif
10921     pozitif
Name: label, dtype: object

In [29]:
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [30]:
train_y[0:10]

array([1, 1, 0, 1, 0, 1, 0, 1, 1, 1])

**Count Vectors**

her satırın içerdiği kelimelerin bir bulunma frekansı vardır, kelimenin varlığına göre 0 ve 1 den oluşan bir array oluşur.

In [31]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer()

In [32]:
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

bakalım buna bi

In [33]:
vectorizer.get_feature_names()[0:10]

['aaa',
 'aaliyah',
 'abagnale',
 'abandon',
 'abandoned',
 'abbass',
 'abbott',
 'abbreviated',
 'abc',
 'abdul']

arrayimize bakalım

In [34]:
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

**TF-IDF**

WordLevel

In [35]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer()

In [36]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

In [37]:
tf_idf_word_vectorizer.get_feature_names()[0:10]

['aaa',
 'aaliyah',
 'abagnale',
 'abandon',
 'abandoned',
 'abbass',
 'abbott',
 'abbreviated',
 'abc',
 'abdul']

**N-Gram Level**

In [38]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(ngram_range=(2, 3))

In [39]:
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

**Characters Level**

In [40]:
tf_idf_ngram_chars_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2,3))
tf_idf_ngram_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

In [41]:
x_train_tf_idf_chars = tf_idf_ngram_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars = tf_idf_ngram_chars_vectorizer.transform(test_x)

## **Sentiment Sınıflandırması**

### Lojistik Regresyon

In [42]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.832897489539749


In [31]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.8289748953974895


In [32]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.7506276150627615


In [33]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7776673640167363


Farklı Vector işlemleriyle alınana farklı doğruluk oranları şimdi başka yöntemlerle deneyelim.

### Naive Bayes

In [34]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.831642259414226


In [35]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.8322175732217574


In [36]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.7710251046025104


In [37]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7529811715481173


### Random Forests

In [38]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8257322175732218


In [39]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.8294456066945607


In [40]:
rf = ensemble.RandomForestClassifier()
rf_model = loj.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.7506276150627615


In [41]:
rf = ensemble.RandomForestClassifier()
rf_model = loj.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7776673640167363


### XGBoost

In [42]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.6226464435146444


In [43]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.6218619246861925


In [44]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.561663179916318


In [45]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.6939330543933054


Her türlü modeli denedik şimdi iyi sonuç veren lojistik regresyon ile deneyelim

In [43]:
loj_model

LogisticRegression()

yeni yorumun model tarafından nasıl tahmin edileceğine bakalım. tabi önce CountVector işleminden geçirmemiz lazım çünkü bizim modelimiz bu şkilde verilerden anlaşıyor.

In [44]:
yeni_yorum = pd.Series("this film is very nice and good i like it")

yeni_yorum1 = pd.Series("no not good look at that shit very bad")

In [45]:
v = CountVectorizer()
v.fit(train_x)
yeni_yorum = v.transform(yeni_yorum)


In [46]:
loj_model.predict(yeni_yorum)

array([1])

In [50]:
v = CountVectorizer()
v.fit(train_x)
yeni_yorum1 = v.transform(yeni_yorum1)


In [51]:
loj_model.predict(yeni_yorum1)

array([0])

bir olumlu bir olumsuz eleştiri ile modelimizi denemiş olduk bir sonraki notebookta ne öğreneceğiz acabaaa